In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
239,AFG,Asia,Afghanistan,2020-08-26,38071.0,1.0,67.429,1397.0,0.0,3.143,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
410,ALB,Europe,Albania,2020-08-26,8759.0,154.0,157.857,259.0,5.0,3.857,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
650,DZA,Africa,Algeria,2020-08-26,42228.0,370.0,397.714,1456.0,10.0,9.286,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-25,France,248685.0,0.0,30551.0,0.0,0.0,0.0
0,2020-08-25,China,89753.0,0.0,4717.0,0.0,0.0,0.0
0,2020-08-25,Italy,261417.0,0.0,35469.0,0.0,0.0,0.0
0,2020-08-25,Spain,408311.0,0.0,28884.0,0.0,0.0,0.0
0,2020-08-25,United States,5830624.0,0.0,178159.0,0.0,0.0,0.0
0,2020-08-25,World,24236223.0,0.0,822401.0,0.0,0.0,0.0
0,2020-08-25,United Kingdom,327803.0,0.0,41475.0,0.0,0.0,0.0
0,2020-08-25,Germany,235562.0,0.0,9286.0,0.0,0.0,0.0
0,2020-08-25,Iran,364165.0,0.0,20961.0,0.0,0.0,0.0
0,2020-08-25,Turkey,260734.0,0.0,6156.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-26,France,248685.0,248158.0,30551.0,30544.0,527.0,7.0
0,2020-08-26,China,89753.0,89752.0,4717.0,4712.0,1.0,5.0
0,2020-08-26,Italy,261417.0,261174.0,35469.0,35445.0,243.0,24.0
0,2020-08-25,Spain,408311.0,412553.0,28884.0,28924.0,-4242.0,-40.0
0,2020-08-26,United States,5830624.0,5779028.0,178159.0,178486.0,51596.0,-327.0
0,2020-08-26,World,24236223.0,23928539.0,822401.0,819756.0,307684.0,2645.0
0,2020-08-26,United Kingdom,327803.0,327798.0,41475.0,41449.0,5.0,26.0
0,2020-08-26,Germany,235562.0,236429.0,9286.0,9280.0,-867.0,6.0
0,2020-08-26,Iran,364165.0,361150.0,20961.0,20776.0,3015.0,185.0
0,2020-08-26,Turkey,260734.0,261194.0,6156.0,6163.0,-460.0,-7.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")